In [25]:
# Import necessary modules
import os
import mysql.connector as connector  # MySQL connector for database interaction
from mysql.connector import errorcode  # Import error codes for exception handling
import pandas as pd
from IPython.display import display  # For displaying DataFrame in Jupyter Notebook

# Load MySQL Credentials from Environment Variables
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")
DB_HOST = os.getenv("DB_HOST", "localhost")  # Default to localhost if not specified

# Establish MySQL Connection
try:
    connection = connector.connect(
        user=DB_USER,  
        password=DB_PASSWORD,  
        database=DB_NAME,
        host=DB_HOST
    )
    print("\nConnected to MySQL database")
except connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Connection failed: Incorrect username or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(f"Error: {err}")
    exit()  # Exit script if connection fails

# Create cursor
cursor = connection.cursor()

# List All Databases
try:
    cursor.execute("SHOW DATABASES")
    databases = cursor.fetchall()
    print("\nList of Databases:")
    for db in databases:
        print(f"- {db[0]}")
except connector.Error as err:
    print(f"Error retrieving databases: {err}")

# List All Tables
try:
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()
    print("\nTables in LittleLemonDB:")
    for table in tables:
        print(f"- {table[0]}")
except connector.Error as err:
    print(f"Error retrieving tables: {err}")

# List All Stored Procedures
try:
    cursor.execute("""
        SELECT ROUTINE_NAME
        FROM INFORMATION_SCHEMA.ROUTINES
        WHERE ROUTINE_TYPE = 'PROCEDURE' AND ROUTINE_SCHEMA = %s
    """, (DB_NAME,))
    procedures = cursor.fetchall()
    
    print("\nStored Procedures in LittleLemonDB:")
    if procedures:
        for proc in procedures:
            print(f"- {proc[0]}")
    else:
        print("No stored procedures found.")
except connector.Error as err:
    print(f"Error retrieving stored procedures: {err}")

# List All Triggers
try:
    cursor.execute("""
        SELECT TRIGGER_NAME
        FROM INFORMATION_SCHEMA.TRIGGERS
        WHERE TRIGGER_SCHEMA = %s
    """, (DB_NAME,))
    triggers = cursor.fetchall()
    
    print("\nTriggers in LittleLemonDB:")
    if triggers:
        for trigger in triggers:
            print(f"- {trigger[0]}")
    else:
        print("No triggers found in LittleLemonDB.")
except connector.Error as err:
    print(f"Error retrieving triggers: {err}")

# Query to Retrieve Customer Details for Orders Over $40
try:
    query = """
        SELECT 
            cd.FirstName, 
            cd.LastName, 
            CONCAT(cd.PhoneNumber, ' - ', cd.Email) AS ContactDetails,
            o.TotalCost
        FROM Customer_Details AS cd
        JOIN Orders AS o ON cd.CustomerID = o.CustomerID
        WHERE o.TotalCost > 40;
    """
    cursor.execute(query)
    results = cursor.fetchall()

    print("\nCustomers Who Spent Over $40:")
    if results:
        for row in results:
            print(f"Name: {row[0]} {row[1]} | Contact: {row[2]} | Bill: ${row[3]}")
    else:
        print("No customers found who spent over $40.")
except connector.Error as err:
    print(f"Error retrieving customers: {err}")
    


Connected to MySQL database

List of Databases:
- information_schema
- LittleLemonDB
- mysql
- performance_schema
- sys

Tables in LittleLemonDB:
- Audit_Log
- Bookings
- Customer_Details
- Menu
- Order_Delivery_Status
- Order_Items
- Orders
- Staff_Information
- Tables

Stored Procedures in LittleLemonDB:
- AddMultipleItemsToOrder
- AddValidBooking
- CancelAllBookingsForDate
- CancelBooking
- ChangeBookingTable
- CheckBooking
- ErrorHandler
- FindAvailableTables
- GetBookingsForDate
- GetMaxQuantity
- UpdateBooking
- UpdateOrderStatus

Triggers in LittleLemonDB:
- log_delete_booking
- log_insert_booking
- log_update_booking
- log_update_order_status
- update_order_total
- log_insert_order_items
- log_update_order_items
- log_delete_order_items
- log_insert_order
- log_new_order_status
- after_order_cancelled

Customers Who Spent Over $40:
Name: Jane Doe | Contact: 555-0002 - jane.doe@example.com | Bill: $100.00
Name: Charlie Brown | Contact: 555-0005 - charlie.brown@example.com | Bil

In [26]:
order_status_query = """
SELECT 
    o.OrderID,
    b.BookingID,
    b.CustomerID,
    CONCAT(c.FirstName, ' ', c.LastName) AS CustomerName,
    o.OrderDate,
    o.StaffID,
    CONCAT(s.FirstName, ' ', s.LastName) AS StaffMember,
    ods.Status AS OrderStatus,
    b.Status AS BookingStatus
FROM Orders o
LEFT JOIN Bookings b ON o.BookingID = b.BookingID
LEFT JOIN Customer_Details c ON b.CustomerID = c.CustomerID
LEFT JOIN Staff_Information s ON o.StaffID = s.StaffID
LEFT JOIN Order_Delivery_Status ods ON o.OrderID = ods.OrderID;
"""

# Execute query
cursor.execute(order_status_query)

# Fetch results and convert them into a DataFrame
df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display DataFrame in Jupyter Notebook
display(df)  # Using Pandas display function

,OrderID,BookingID,CustomerID,CustomerName,OrderDate,StaffID,StaffMember,OrderStatus,BookingStatus
0,152,NaN,NaN,None,2025-03-14,7,John Doe,Cancelled,None
1,153,146.0,10.0,Jane Doe,2025-03-15,7,John Doe,Delivered,New Booking
2,154,147.0,11.0,Bob Smith,2025-03-15,7,John Doe,Delivered,New Booking
3,155,148.0,12.0,Alice Johnson,2025-03-15,7,John Doe,Delivered,New Booking
4,156,149.0,13.0,Charlie Brown,2025-03-15,7,John Doe,Delivered,New Booking
5,157,150.0,14.0,David Taylor,2025-03-15,7,John Doe,Delivered,New Booking
6,158,151.0,15.0,Emily White,2025-03-15,7,John Doe,Delivered,New Booking
7,159,152.0,16.0,Frank Miller,2025-03-15,7,John Doe,Delivered,New Booking
8,160,153.0,17.0,Grace Wilson,2025-03-15,7,John Doe,Delivered,New Booking
9,161,154.0,18.0,Hannah Moore,2025-03-15,7,John Doe,Delivered,New Booking


In [27]:
# Define the SQL query to fetch all bookings
all_bookings_query = """
SELECT 
    BookingID, 
    CustomerID, 
    BookingDate, 
    Status 
FROM Bookings;
"""

# Execute query
cursor.execute(all_bookings_query)

# Fetch all records and convert to Pandas DataFrame
import pandas as pd
from IPython.display import display

df_bookings = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display DataFrame in Jupyter Notebook
print("\n📌 Bookings:")
display(df_bookings)


📌 Bookings:


,BookingID,CustomerID,BookingDate,Status
0,146,10,2025-03-15,New Booking
1,147,11,2025-03-15,New Booking
2,148,12,2025-03-15,New Booking
3,149,13,2025-03-15,New Booking
4,150,14,2025-03-15,New Booking
5,151,15,2025-03-15,New Booking
6,152,16,2025-03-15,New Booking
7,153,17,2025-03-15,New Booking
8,154,18,2025-03-15,New Booking
9,155,9,2025-03-15,New Booking


In [28]:
# Query to retrieve staff information
staff_query = """
SELECT 
    StaffID, 
    FirstName, 
    LastName, 
    Role 
FROM Staff_Information;
"""
cursor.execute(staff_query)
df_staff = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Query to retrieve customer information
customer_query = """
SELECT 
    CustomerID, 
    FirstName, 
    LastName, 
    PhoneNumber, 
    Email 
FROM Customer_Details;
"""
cursor.execute(customer_query)
df_customers = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display staff and customer details
print("\n📌 Staff Information:")
display(df_staff)

print("\n📌 Customer Information:")
display(df_customers)


📌 Staff Information:


,StaffID,FirstName,LastName,Role
0,7,John,Doe,Waiter
1,8,Jane,Smith,Chef
2,9,Bob,Johnson,Manager
3,10,Alice,Williams,Waiter
4,11,Charlie,Brown,Chef
5,12,David,Taylor,Cleaner
6,13,Emily,White,Waiter
7,14,Frank,Miller,Manager
8,15,Grace,Wilson,Waiter
9,16,Hannah,Moore,Chef



📌 Customer Information:


,CustomerID,FirstName,LastName,PhoneNumber,Email
0,9,John,Doe,555-0001,john.doe@example.com
1,10,Jane,Doe,555-0002,jane.doe@example.com
2,11,Bob,Smith,555-0003,bob.smith@example.com
3,12,Alice,Johnson,555-0004,alice.johnson@example.com
4,13,Charlie,Brown,555-0005,charlie.brown@example.com
5,14,David,Taylor,555-0006,david.taylor@example.com
6,15,Emily,White,555-0007,emily.white@example.com
7,16,Frank,Miller,555-0008,frank.miller@example.com
8,17,Grace,Wilson,555-0009,grace.wilson@example.com
9,18,Hannah,Moore,555-0010,hannah.moore@example.com


In [34]:
# Query to retrieve menu items
menu_query = "SELECT * FROM Menu;"
cursor.execute(menu_query)
df_menu = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Query to retrieve tables
tables_query = "SELECT * FROM Tables;"  # Adjust the table name if needed
cursor.execute(tables_query)
df_tables = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display the menu in Jupyter Notebook
print("\n📌 Menu Items:")
display(df_menu)

print("\n📌Restaurant Tables:")
display(df_tables)


📌 Menu Items:


,MenuID,ItemName,Category,Price
0,11,Spaghetti Bolognese,Courses,15.00
1,12,Caesar Salad,Starters,10.00
2,13,Cheeseburger,Courses,12.00
3,14,Fried Chicken,Courses,14.00
4,15,Margarita Pizza,Courses,12.50
5,16,Penne Arrabbiata,Courses,13.00
6,17,Steak,Courses,25.00
7,18,Grilled Salmon,Courses,22.00
8,19,Lemonade,Drinks,4.00
9,20,Iced Tea,Drinks,3.50



📌Restaurant Tables:


,TableID,TableNumber,Capacity,IsAvailable,LastUpdated
0,1,1,4,1,2025-03-12 23:27:34
1,2,2,4,1,2025-03-12 23:27:34
2,3,3,2,1,2025-03-12 23:27:34
3,4,4,2,1,2025-03-12 23:27:34
4,5,5,4,1,2025-03-12 23:27:34
5,6,6,4,1,2025-03-12 23:27:34
6,7,7,2,1,2025-03-12 23:27:34
7,8,8,2,1,2025-03-12 23:27:34
8,9,9,4,1,2025-03-12 23:27:34
9,10,10,4,1,2025-03-12 23:27:34


Your project will be evaluated according to the following grading criteria:

- Was the GetMaxQuantity() procedure properly implemented?
- Was the ManageBooking() procedure properly implemented?
- Was the UpdateBooking() procedure properly implemented?
- Was the AddBooking() procedure properly implemented?
- Was the CancelBooking() procedure properly implemented?

In [39]:
# GetMaxQuantity - This procedure gets the maximum quantity ordered from the Order_Items table
# Call the stored procedure GetMaxQuantity
try:
    cursor.callproc('GetMaxQuantity')

    # Fetch results
    for result in cursor.stored_results():
        max_quantity = result.fetchone()[0]  # Fetch first row & column
        print(f"\nMaximum Quantity Ordered in Order_Items Table: {max_quantity}")

except connector.Error as err:
    print(f"Error executing GetMaxQuantity(): {err}")


Maximum Quantity Ordered in Order_Items Table: The maximum quantity ordered is: 3


The following code covers several procedures that fall within the remit of ManageBooking()

In [40]:
# full process of making a booking, creating an order, adding menu items to the order and then updating status of order to delivered.
# we will also cover some booking cancellations

# Test Data for 5 Bookings with today's date (2025-03-20)
bookings = [
    ("2025-03-20", 2, 10, "Admin"), #BookingDate, TableID, CustomerID, PerformedBy
    ("2025-03-20", 4, 11, "Admin"),
    ("2025-03-20", 6, 12, "Admin"),
    ("2025-03-20", 8, 13, "Admin"),
    ("2025-03-20", 10, 14, "Admin")
]

# Execute AddValidBooking() for each booking
try:
    for booking in bookings:
        cursor.callproc('AddValidBooking', booking)
    
    connection.commit()  # Commit transaction after all insertions
    print("\n5 new bookings successfully added for 2025-03-20.")

except connector.Error as err:
    print(f"Error executing AddValidBooking(): {err}")


5 new bookings successfully added for 2025-03-20.


In [45]:
# Retrieve the newly added bookings for 2025-03-20
bookings_query = """
SELECT * 
FROM Bookings 
WHERE BookingDate = "2025-03-20";
"""

cursor.execute(bookings_query)
df_bookings = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display the bookings for 2025-03-20
print("\n📌 Bookings for 2025-03-20:")
display(df_bookings)


📌 Bookings for 2025-03-20:


,BookingID,CustomerID,BookingDate,TableID,Status
0,195,10,2025-03-20,2,New Booking
1,196,11,2025-03-20,4,New Booking
2,197,12,2025-03-20,6,New Booking
3,198,13,2025-03-20,8,New Booking
4,199,14,2025-03-20,10,New Booking


In [47]:
#Close cursor and connection
cursor.close()
connection.close()
print("\n✅ Connection closed.")


✅ Connection closed.
